# 3DRDNN

Main pipeline for research in my Master.

## goals in this notebook

Prepare the pipeline for any 3d DNN to train on CT data.
1) data loader
2) DNN
3) Training
4) Results comparison

In [21]:
# Testing reading GPU
from distutils.version import LooseVersion
import warnings
import tensorflow as tf
from matplotlib import pyplot as plt
import numpy as np
# Check TensorFlow Version
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please ensure you have installed TensorFlow correctly')
else:
    print(tf.test.gpu_device_name())

TensorFlow Version: 2.8.0
/device:GPU:0


In [22]:
# Data loader
from utils import data_loader

path_train = "data\LITS_Challenge\Training_Batch_2"
path_valid = "data\LITS_Challenge\Training_Batch_1"

loader_train = data_loader.NiiDataLoader(path_train)
loader_valid = data_loader.NiiDataLoader(path_valid)

train_generator = loader_train.data_generator_3d_lesion_chunks()
valid_generator = loader_valid.data_generator_3d_lesion_chunks()





#counter_train = loader_train.generator_data_len()
#counter_valid = loader_valid.generator_data_len()
"""counter_train = 0
counter_valid = 0

for x in train_generator:
    counter_train += 1

for x in valid_generator:
    counter_valid += 1"""

counter_train = 14635
counter_valid = 14918

print(counter_train)
print(counter_valid)

initalised with path data\LITS_Challenge\Training_Batch_2
files: 103,103
initalised with path data\LITS_Challenge\Training_Batch_1
files: 28,28
14635
14918


In [23]:
### learning rate schedule

def scheduler(epoch, lr):
    if epoch < 5:
        lr = 0.001
        return lr
    else:
        return lr * tf.math.exp(-0.05)

In [24]:
#datasets for tf
from data_preprocessing_3d import get_dataset_large

batch_size = 1
dataset = get_dataset_large("D:\\tfrecords_train")

for sample in dataset.take(1):
    print(sample[0].shape)
    print(sample[1].shape)

dataset = dataset.repeat(101).batch(batch_size)
valid_dataset = get_dataset_large("D:\\tfrecords_valid")
valid_dataset = valid_dataset.repeat(100).batch(batch_size)#.shuffle(1000,  reshuffle_each_iteration=True)


print(dataset)
print(valid_dataset)

['D:\\tfrecords_train\\images0.tfrecords', 'D:\\tfrecords_train\\images1.tfrecords', 'D:\\tfrecords_train\\images10.tfrecords', 'D:\\tfrecords_train\\images11.tfrecords', 'D:\\tfrecords_train\\images12.tfrecords', 'D:\\tfrecords_train\\images13.tfrecords', 'D:\\tfrecords_train\\images14.tfrecords', 'D:\\tfrecords_train\\images15.tfrecords', 'D:\\tfrecords_train\\images16.tfrecords', 'D:\\tfrecords_train\\images17.tfrecords', 'D:\\tfrecords_train\\images18.tfrecords', 'D:\\tfrecords_train\\images19.tfrecords', 'D:\\tfrecords_train\\images2.tfrecords', 'D:\\tfrecords_train\\images20.tfrecords', 'D:\\tfrecords_train\\images21.tfrecords', 'D:\\tfrecords_train\\images22.tfrecords', 'D:\\tfrecords_train\\images23.tfrecords', 'D:\\tfrecords_train\\images24.tfrecords', 'D:\\tfrecords_train\\images25.tfrecords', 'D:\\tfrecords_train\\images26.tfrecords', 'D:\\tfrecords_train\\images27.tfrecords', 'D:\\tfrecords_train\\images28.tfrecords', 'D:\\tfrecords_train\\images29.tfrecords', 'D:\\tfrecord

In [25]:
# Network set-up
from utils import models

model = models.model_call(model_name="3DUNET",pxz=32,px=256,features=4)

In [26]:
# Metrics and training
 
adam = tf.keras.optimizers.Adam(
    learning_rate=0.00001, beta_1=0.9, beta_2=0.999, epsilon=1e-07, amsgrad=False,
    name='Adam')
precision_all = tf.keras.metrics.Precision(
    thresholds=0.5)
precision = tf.keras.metrics.Precision(
    thresholds=0.5, class_id=1)
recall = tf.keras.metrics.Recall(
    thresholds=0.5, class_id=1)

model.compile(loss="categorical_crossentropy", optimizer=adam, metrics=[precision_all,precision,recall])
model.summary(positions=[.33, .66, .78, 1.])

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape                    Param #     Connected to          
 input_3 (InputLayer)           [(None, 32, 256, 256, 1)]       0           []                    
                                                                                                  
 conv3d_30 (Conv3D)             (None, 32, 256, 256, 4)         112         ['input_3[0][0]']     
                                                                                                  
 batch_normalization_24 (BatchN  (None, 32, 256, 256, 4)        16          ['conv3d_30[0][0]']   
 ormalization)                                                                                    
                                                                                                  
 conv3d_31 (Conv3D)             (None, 32, 256, 256, 8)         296         ['batch_normaliz

In [27]:
# Results
# reading checkpoint if needen
#
#################################

from tensorflow.keras.callbacks import (
    EarlyStopping,
    ModelCheckpoint,
    ReduceLROnPlateau,
    TerminateOnNaN,
)

# training
callbacks = [
    # EarlyStopping(patience=10, verbose=1),
    #ReduceLROnPlateau(factor=0.1, patience=10, min_lr=0.0000001, verbose=1),
    tf.keras.callbacks.LearningRateScheduler(scheduler),
    ModelCheckpoint(
        "models\\3DUNET_lession_16_32_512\\epoch-{epoch:02d}-valloss-{val_loss:.4f}-{val_precision_1:.2f}-{val_recall:.2f}.hdf5",
        verbose=1,
        save_best_only=False,
        save_weights_only=True,
    ),
    TerminateOnNaN(),
]
epochs = 60
steps_per_epoch = (
    counter_train / batch_size
) 


validations_steps = counter_valid / batch_size

history = model.fit(
    dataset,
    validation_data=valid_dataset,
    callbacks=callbacks,
    initial_epoch=0,
    epochs=epochs,steps_per_epoch = 30*25,
    validation_steps=11*25,
)

Epoch 1/60
669/750 [=========================>....] - ETA: 11s - loss: 0.0355 - precision_4: 0.9315 - precision_5: 0.0537 - recall_2: 0.2037

DataLossError: Graph execution error:

Detected at node 'IteratorGetNext' defined at (most recent call last):
    File "c:\Users\kaczm\.conda\envs\tf2.8\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "c:\Users\kaczm\.conda\envs\tf2.8\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "c:\Users\kaczm\.conda\envs\tf2.8\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "c:\Users\kaczm\.conda\envs\tf2.8\lib\site-packages\traitlets\config\application.py", line 992, in launch_instance
      app.start()
    File "c:\Users\kaczm\.conda\envs\tf2.8\lib\site-packages\ipykernel\kernelapp.py", line 711, in start
      self.io_loop.start()
    File "c:\Users\kaczm\.conda\envs\tf2.8\lib\site-packages\tornado\platform\asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "c:\Users\kaczm\.conda\envs\tf2.8\lib\asyncio\base_events.py", line 595

: 

In [8]:


plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

np.save("2Dunet_history_lesion.npy",np.array(history))



NameError: name 'history' is not defined

In [ ]:
#a zrób testowy trening od modelu do wątroby :))) mnoże będzie lepiej!

In [12]:
new_model = tf.keras.models.load_model("models\\2DUNET_lesion_16_256\0.0023-0.43-0.76-epoch-74.hdf5")


n_max = 700
n=0
for x in loader_valid.data_generator_2d_lesion():
   if n < 100:
      pass
   elif n < n_max:
      plt.subplot(1,3,1)
      plt.imshow(x[0])
      plt.subplot(1,3,2)
      preds = new_model.predict(tf.reshape(x[0],[1,256,256,1]))
      plt.imshow(preds[0,:,:,1])
      plt.subplot(1,3,3)
      plt.imshow(x[1][:,:,1])
      plt.show()
   else:
      break
   n+=1